In [4]:
import os
import sys
import pickle
import json
import re

In [37]:
def dedup(lst):
    return [i for n, i in enumerate(lst) if i not in lst[n + 1:]]

def read_log(fp):
    examples = []
    with open(fp, 'r') as f:
        entry = f.readline()
        while entry:
            m = re.search('"FILTER_RESULTS",{"type":"FILTER_RESULTS","payload":(.+)}', entry)
            if m:
                json_str = m.group(1)
                exs = json.loads(json_str)
                for ex in exs['examples']:
                    ex.pop("id", None)
                examples.append(exs['examples'])
                
            entry = f.readline()
    
    return examples

In [38]:
def read_log_dir(path):
    examples = []
    for f in os.listdir(path):
        examples += read_log(os.path.join(path, f))
        
    return dedup(examples)

In [51]:
examples = read_log_dir('/Users/aaron/Downloads/Website Logs')

In [54]:
import requests

def run_query(exs):
    response = requests.post('http://hoogleplus.goto.ucsd.edu/api/search/example', json={"facts": exs})
    return response.json()

In [57]:
tests = []
for exs in examples:
    if len(exs) == 0:
        continue
    print(exs)
    res = run_query(exs)
    if res['error']:
        tests.append((exs, res['error']))
    else:
        tests.append((exs, res['typeCandidates']))

[{'inputs': ['"cccbbbaa"'], 'output': '"cba"'}, {'inputs': ['"aaabbab"'], 'output': '"abab"'}]
[{'inputs': ['[0]'], 'output': '[]'}]
[{'inputs': ['"aaabbab"'], 'output': '"abab"'}, {'inputs': ['[0]'], 'output': '[]'}]
[{'inputs': ['[]'], 'output': '[]'}, {'inputs': ['[0]'], 'output': '[0]'}, {'inputs': ['[1]'], 'output': '[1]'}]
[{'inputs': ['[]'], 'output': '[]'}, {'inputs': ['[0]'], 'output': '[0]'}, {'inputs': ['[1]'], 'output': '[1]'}, {'inputs': ['[-1]'], 'output': '[-1]'}, {'inputs': ['[2]'], 'output': '[2]'}]
[{'inputs': ['[(\\x -> x + 2), (\\x -> x * 2)]', '5'], 'output': '[7, 10]'}, {'inputs': ['[]', '0'], 'output': '[]'}]
[{'inputs': ['2', '"1"'], 'output': '"11"'}]
[{'inputs': ['4', '"abc"'], 'output': '"abcabcabcabc"'}]
[{'inputs': ['[]'], 'output': '[]'}]
[{'inputs': ['1', '"abc"'], 'output': '"abc"'}, {'inputs': ['4', '"A"'], 'output': '""'}]
[{'inputs': ['1', '"abc"'], 'output': '"abc"'}]
[{'inputs': ['1', '2', '3'], 'output': '5'}]
[{'inputs': ['\\x -> x * 2', '3', '1']

[{'inputs': ['"Nadia Polikarpova"'], 'output': '"N.P"'}, {'inputs': ['"Kasra Ferdowsi Fard"'], 'output': '"K.F.F"'}]
[{'inputs': ['0', '[Nothing, Just 3, Just 4]'], 'output': '3'}, {'inputs': ['0', '[Nothing, Nothing]'], 'output': '0'}, {'inputs': ['0', '[Nothing, Just 1]'], 'output': '1'}]
[{'inputs': ['1', '[Just 5, Nothing]'], 'output': '5'}, {'inputs': ['0', '[Nothing, Just 3, Just 4]'], 'output': '3'}, {'inputs': ['0', '[Nothing, Nothing]'], 'output': '0'}, {'inputs': ['0', '[Nothing, Just 1]'], 'output': '1'}]
[{'inputs': ['(\\x -> x ++ x)', '3', '"f-'], 'output': '"f-f-f-f-f-f-f-f-"'}]
[{'inputs': ['(\\x -> x + 1)', '3', '0'], 'output': '3'}]
[{'inputs': ['(\\x -> "")', '3', '0'], 'output': '3'}]
[{'inputs': ['(\\x -> 0.01)', '3', '0'], 'output': '3'}]
[{'inputs': ['0', '[Nothing, Just 1]'], 'output': '1'}]
[{'inputs': ['(a -> b)', 'Num', 'a'], 'output': '[b]'}]
[{'inputs': ['(\\x -> x ++ x)', '3', '"f-"'], 'output': '"f-f-f-f-f-f-f-f-"'}]
[{'inputs': ['2', '"abc"'], 'output': '

In [65]:
import csv

def write_csv(tests):
    with open('inference_lines.csv', 'w', newline='') as f:
        test_writer = csv.writer(f, delimiter='\t',dialect=csv.excel)
        for (exs, res) in tests:
            test_writer.writerow(('\r\n'.join(map(lambda x: str(x), exs)), '\r\n'.join(res)))
        

In [66]:
write_csv(tests)